## Equipo 48

| Nombre | Matrícula |
| ------ | --------- |
| André Zaragoza  | A01797076 |
| Héctor Santillán | A01633395 |
| Pablo de Jesus González | A01321850 |
| Delbert Custodio | A01795613 |
| Abel Diaz | A00566705 |

# Dataset a trabajar: Seoul Bike Sharing Demand

El dataset conocido como `Seoul Bike Sharing Demand` es una serie de registros los cuales fueron recabados en Febrero de 2020. El dataset contiene 13 columnas y 8760 registros en su formato original.

Las columnas o features que se encuentran en el dataset son los siguientes:

| Feature | Tipo | Notas |
| ------- | ---- | ----- |
| Date    | Temporal | Contiene la fecha en que se llevó a cabo un registro. Su formato es DD-MM-YYYY. |
| Rented Bike Count | Entero | Muestra la cantidad de bicicletas que se rentaron a cierta hora en cierta fecha. |
| Hour    | Temporal | Es la hora del día en que se registraron la cantidad de bicicletas rentadas respecto a la hora anterior. |
| Temperature(°C) | Continua | Es la temperatura en grados Celsius que se registró en cierta fecha y hora del ambiente. |
| Humidity(%) | Entero | Es la humedad relativa del ambiente (en %) registrada en la fecha y hora del registro. |
| Wind speed (m/s) | Continua | La velocidad del viento en m/s registrada en la fecha y hora correspondientes. |
| Visibility (10m) | Entero | Visibilidad en factores de 10 metros. |
| Dew point temperature(°C) | Continua | Es la temperatura en la cual el aire se satura de humedad. |
| Solar Radiation (MJ/m²)  | Continua | Cantidad o medida de radiación solar absorbida por unidad de área. | 
| Rainfall(mm) | Continua | Cantidad de lluvia registrada en milimetros. |
| Snowfall (cm) | Continua | Cantidad de nieve registrada, en milimetros. | 
| Seasons | Categorica, nominal. | La época o temporada del año. |
| Holiday | Binaria o Booleana | Indica si la fecha registada es una festividad. |
| Functioning Day | Binaria. | Indica si el servicio de renta de bicicletas operó o no en la fecha indicada. |

<br>
<br>

Respecto al dataset modificado, podemos encontrar una columna extra:

| Feature | Tipo | Notas |
| ------- | ---- | ----- |
| mixed_type_col | Desconocida | Debemos analizar la columna en nuestro EDA. |

# Problemática a resolver

Según la página de UCI sobre el dataset original, se introdujo la posibilidad de rentar bicicletas en distintas ciudades para mejorar la movilidad dentro de las zonas urbanas. Con esto, un aspecto importante es poder suplir la demanda de bicicletas que se puedan rentar a cualquier hora del día, ya que:

- El esperar mucho tiempo para rentar una bicicleta puede causar que el uso de éstas se reduzca.
- Utilizar una bicicleta para moverse dentro de una ciudad reduce el tráfico vehicular para evitar congestionamientos, por lo que es un indispensable cubrir la demanda de bicicletas que se necesiten durante el día y sobre todo a horas pico.

Por lo anterior, el dataset produce una base sólida para trabajar un modelo de regresión que pueda predecir la demanda de bicicletas que se tendrá por cada hora del día. Esta predicción ayudará a que las autoridades puedan mantener una oferta de bicicletas acorde a las necesidades de la población, y a planificar de mejor manera la cantida de bicicletas a adquirir y dar mantenimiento.

Por lo tanto, la variable a predecir para resolver la problemática es `Rented_Bike_Count`.

# Inicialización y Carga de Datos 

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 1. Cargar el Dataset
df = pd.read_csv('./csv/seoul_bike_sharing_modified.csv')
print("Dataset cargado exitosamente. Las primeras 5 filas:")

df.head(5)

Dataset cargado exitosamente. Las primeras 5 filas:


,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,mixed_type_col
0,01/12/2017,254.0,0.0,-5.2,37.0,2.2,2000.0,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,876
1,01/12/2017,204.0,1.0,-5.5,38.0,0.8,2000.0,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,798
2,01/12/2017,173.0,2.0,-6.0,39.0,1.0,2000.0,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,231
3,01/12/2017,107.0,3.0,-6.2,40.0,0.9,2000.0,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,bad
4,01/12/2017,78.0,4.0,-6.0,36.0,2.3,2000.0,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,536


In [17]:
# 2. Renombrar y Estandarizar Columnas
# Eliminaremos los espacios entre los nombres de los features para reemplazarlos por underscores (_)

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.rename(columns={'rented_bike_count': 'demanda'}, inplace=True)

# Imprimimos las nuevas columnas
print('Columnas a trabajar:\n')
df.dtypes

Columnas a trabajar:



date                         object
demanda                      object
hour                         object
temperature(°c)              object
humidity(%)                  object
wind_speed_(m/s)             object
visibility_(10m)             object
dew_point_temperature(°c)    object
solar_radiation_(mj/m2)      object
rainfall(mm)                 object
snowfall_(cm)                object
seasons                      object
holiday                      object
functioning_day              object
mixed_type_col               object
dtype: object

En este punto es posible darnos cuenta de que contamos con un dataset el cual:

- Contiene "ruido" en las variables de tipo numérico.
- Variablles o features que deberían ser binarios contienen valores extraños (`holiday` y `functioning_day` son un claro ejemplo de ésto).

# Limpieza de nuestro dataset

### Para `holiday`  y `functioning_day`

Notamos que los valores válidos para estos features son los siguientes:

- `Para holiday`: `No holiday` & `Holiday`.
- `Para functioning_day`: `Yes` & `No`.

In [29]:
print( f"Valores únicos para `holiday`: {df['holiday'].unique()}" )
print( f"Valores únicos para `functioning_day`: {df['functioning_day'].unique()}" )

Valores únicos para `holiday`: ['No Holiday' ' nO hOLIDAY ' nan ' hOLIDAY ' 'Holiday' ' NAN ']
Valores únicos para `functioning_day`: ['Yes' ' yES ' ' NAN ' nan 'No' ' nO ']


#### Para `holiday`

Por lo tanto, ahora procedemos a trabajar sobre `Holiday` para asegurarnos que los valores en nuestro dataset sean correctos.

In [27]:
def correct_holiday_values( value_to_clean ):
    
    
    if ( (value_to_clean == 'nan') or (value_to_clean is None ) or (value_to_clean == '')):
        return None
    else:
        value_to_clean = str(value_to_clean).lower()
        value_to_clean = value_to_clean.strip()
    
        if value_to_clean == 'holiday':
            return 'yes'
        elif value_to_clean == 'no holiday':
            return 'no'
        
df['Holiday_or_not'] = df['holiday'].apply(correct_holiday_values)
print( f"Al aplicar nuestra función de limpieza tenemos los siguientes valores únicos en `holiday`: {df['Holiday_or_not'].unique()}")

Al aplicar nuestra función de limpieza tenemos los siguientes valores únicos en `holiday`: ['no' None 'yes']


In [ ]:
# Copiamos los valores obtenidos para holiday y eliminamos la columna que usamos para limpiar esta data

df['holiday'] = df['Holiday_or_not']
df.drop(columns=['Holiday_or_not'], inplace=True)


In [45]:
# Ahora revisamos qué fechas tienen valores None en la columna de holiday para investigar si estos fueron días festivos en Seoul.

print( df.loc[df['holiday'].isna(), 'date'].unique() )

[' 10/12/2017 ' '17/12/2017' '19/12/2017' '20/12/2017' '21/12/2017'
 '22/12/2017' '23/12/2017' '26/12/2017' '29/12/2017' '30/12/2017'
 '05/01/2018' '09/01/2018' '14/01/2018' '19/01/2018' '21/01/2018'
 '23/01/2018' '27/02/2018' '28/02/2018' '01/03/2018' '18/03/2018'
 '20/03/2018' '21/03/2018' '23/03/2018' '09/04/2018' '16/04/2018' nan
 '26/04/2018' '27/04/2018' '02/05/2018' '06/05/2018' '19/05/2018'
 '20/05/2018' '22/05/2018' '23/05/2018' ' 27/05/2018 ' '11/06/2018'
 '12/06/2018' '18/06/2018' '19/06/2018' ' 25/06/2018 ' '08/07/2018'
 '11/07/2018' '13/07/2018' '24/07/2018' ' 25/07/2018 ' '30/07/2018'
 '31/07/2018' '03/08/2018' '13/08/2018' '22/08/2018' '31/08/2018'
 '02/09/2018' '05/09/2018' '14/09/2018' ' 20/09/2018 ' '22/09/2018'
 '23/09/2018' '30/09/2018' '05/10/2018' '15/10/2018' '27/10/2018'
 '28/10/2018' '31/10/2018' '02/11/2018' '06/11/2018' '12/11/2018'
 '13/11/2018' '14/11/2018' '22/11/2018' '23/11/2018' '25/11/2018'
 '30/11/2018' '15/11/2018' '28/04/2018']


Y luego de una búsqueda rápida en internet, las siguientes fechas fueron festivas en Seoul:

- 01/03/2018
- 22/05/2018

In [49]:
# Entonces, procedemos a cambiar los valores None por 'yes' o 'no' con base en la fecha de registro.
# En este paso, existe un registro con fecha NaN que tomará el valor de 'no', pero esto podemos trabajarlo después.

festive_dates = ['01/03/2018', '22/05/2018']

df['holiday'] = [ 
                 'yes' if  (h is None and d in festive_dates)
                 else 'no' if (h is None and d not in festive_dates)
                 else h for d,h in zip(df['date'], df['holiday'])]

df['holiday'].unique()

array(['no', 'yes'], dtype=object)

#### Para `functioning_day`

Al correr el código de abajo es posible notar que no se encontró algún valor None o vacío en la columna `functioning_day` al usar una función similar al procesamiento de `holiday`, por lo que en este paso procedemos solamente a modificar los valores de `functioning_day` según lo obtenido por nuestra función.

In [51]:
def correct_functioning_day_values( value_to_clean):
    
    if ( (value_to_clean == 'nan') or (value_to_clean is None ) or (value_to_clean == '')):
        return None
    
    else:
        
        value_to_clean = str(value_to_clean).lower()
        value_to_clean = value_to_clean.strip()
    
        if value_to_clean == 'yes':
            return 'yes'
        else:
            return 'no'
        
df['functioning_day_or_not'] = df['functioning_day'].apply(correct_functioning_day_values)
print( f"Al aplicar nuestra función de limpieza tenemos los siguientes valores únicos en `functioning_day`: {df['functioning_day_or_not'].unique()}")

Al aplicar nuestra función de limpieza tenemos los siguientes valores únicos en `functioning_day`: ['yes' 'no']


In [52]:
df['functioning_day'] = df['functioning_day_or_not']
df.drop(columns=['functioning_day_or_not'], inplace=True)
df['functioning_day'].unique()

array(['yes', 'no'], dtype=object)

Y ahora procedemos a revisar la distribución de cada 

In [ ]:
# Convertir 'date' al tipo datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Combinar fecha y hora en una sola columna para análisis de series de tiempo
df['datetime'] = df['date'] + pd.to_timedelta(df['hour'], unit='h')
df.set_index('datetime', inplace=True)
df.drop(['date', 'hour'], axis=1, inplace=True)

print("\n--- DataFrame después de la conversión de tipos ---")
print(df.head())

# Exploratory Data Analysis (EDA)

In [15]:
print("\n--- Información General y Tipos de Datos ---")
df.info()

print("\n\n--- Estadísticas Descriptivas de Variables Numéricas ---")
print(df.describe().T)

print("\n--- Conteo de Valores Únicos en Variables Categóricas/Discretas ---")
print(df['seasons'].value_counts())
print(df['holiday'].value_counts())
print(df['functioning_day'].value_counts())


--- Información General y Tipos de Datos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8935 entries, 0 to 8934
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   date                       8857 non-null   object
 1   demanda                    8829 non-null   object
 2   hour                       8820 non-null   object
 3   temperature(°c)            8846 non-null   object
 4   humidity(%)                8837 non-null   object
 5   wind_speed_(m/s)           8812 non-null   object
 6   visibility_(10m)           8843 non-null   object
 7   dew_point_temperature(°c)  8833 non-null   object
 8   solar_radiation_(mj/m2)    8846 non-null   object
 9   rainfall(mm)               8855 non-null   object
 10  snowfall_(cm)              8834 non-null   object
 11  seasons                    8856 non-null   object
 12  holiday                    8857 non-null   object
 13  functioning_day  

# Limpieza y Transformación de Datos


--- DataFrame después de la conversión de tipos ---
                     demanda  temperature(°c)  humidity(%)  wind_speed_(m/s)  \
datetime                                                                       
2017-12-01 00:00:00      254             -5.2           37               2.2   
2017-12-01 01:00:00      204             -5.5           38               0.8   
2017-12-01 02:00:00      173             -6.0           39               1.0   
2017-12-01 03:00:00      107             -6.2           40               0.9   
2017-12-01 04:00:00       78             -6.0           36               2.3   

                     visibility_(10m)  dew_point_temperature(°c)  \
datetime                                                           
2017-12-01 00:00:00              2000                      -17.6   
2017-12-01 01:00:00              2000                      -17.6   
2017-12-01 02:00:00              2000                      -17.7   
2017-12-01 03:00:00              2000         

In [10]:
# 1. Verificar Nulos (Métricas)
print("\n--- Conteo de Valores Nulos por Columna ---")
print(df.isnull().sum())
# Si aparecen nulos, la acción recomendada (imputación con media/mediana o eliminación) depende de la cantidad.

# 2. Manejo de Inconsistencias
# Las variables 'functioning_day' deben ser binarias (Yes/No o Fun/NoFunc)
# En este dataset son 'Yes' y 'No', lo cual es consistente.

# Convertir 'functioning_day' y 'holiday' a numérico (0 y 1)
df['functioning_day'] = df['functioning_day'].map({'Yes': 1, 'No': 0})
df['holiday'] = df['holiday'].map({'No Holiday': 0, 'Holiday': 1})


--- Conteo de Valores Nulos por Columna ---
demanda                      0
temperature(°c)              0
humidity(%)                  0
wind_speed_(m/s)             0
visibility_(10m)             0
dew_point_temperature(°c)    0
solar_radiation_(mj/m2)      0
rainfall(mm)                 0
snowfall_(cm)                0
seasons                      0
holiday                      0
functioning_day              0
dtype: int64


In [11]:
# Aplicar la detección de outliers en la variable 'demanda'
Q1 = df['demanda'].quantile(0.25)
Q3 = df['demanda'].quantile(0.75)
IQR = Q3 - Q1

limite_superior = Q3 + 1.5 * IQR

# 1. Conteo de Outliers
outliers_count = df[df['demanda'] > limite_superior].shape[0]
print(f"\n--- Detección de Outliers en Demanda ---")
print(f"Límite Superior (1.5*IQR): {limite_superior:.0f}")
print(f"Número de outliers detectados: {outliers_count}")

# 2. Opción de Limpieza (Imputación por tope o 'Capping')
# En lugar de eliminar, reemplazamos los outliers con el límite superior (Capping).
# Esto es común en modelos de regresión para mantener el tamaño del dataset.
df['demanda_limpia'] = np.where(
    df['demanda'] > limite_superior,
    limite_superior,
    df['demanda']
)

# Comparación del máximo antes y después
print(f"Máximo de Demanda (Original): {df['demanda'].max()}")
print(f"Máximo de Demanda (Limpia): {df['demanda_limpia'].max():.0f}")

# Se puede eliminar la columna original 'demanda' si solo se usará la 'demanda_limpia'
df.drop('demanda', axis=1, inplace=True)
df.rename(columns={'demanda_limpia': 'demanda'}, inplace=True)


--- Detección de Outliers en Demanda ---
Límite Superior (1.5*IQR): 2377
Número de outliers detectados: 158
Máximo de Demanda (Original): 3556
Máximo de Demanda (Limpia): 2377


### Buscamos algunas relaciones

# Transformaciones Finales (Feature Engineering)

In [8]:
# Aplicar One-Hot Encoding a la variable 'seasons'
df_encoded = pd.get_dummies(df, columns=['seasons'], prefix='season')

print("\n--- DataFrame después del One-Hot Encoding de 'seasons' ---")
print(df_encoded.head())


--- DataFrame después del One-Hot Encoding de 'seasons' ---
                     temperature(°c)  humidity(%)  wind_speed_(m/s)  \
datetime                                                              
2017-12-01 00:00:00             -5.2           37               2.2   
2017-12-01 01:00:00             -5.5           38               0.8   
2017-12-01 02:00:00             -6.0           39               1.0   
2017-12-01 03:00:00             -6.2           40               0.9   
2017-12-01 04:00:00             -6.0           36               2.3   

                     visibility_(10m)  dew_point_temperature(°c)  \
datetime                                                           
2017-12-01 00:00:00              2000                      -17.6   
2017-12-01 01:00:00              2000                      -17.6   
2017-12-01 02:00:00              2000                      -17.7   
2017-12-01 03:00:00              2000                      -17.6   
2017-12-01 04:00:00              

In [9]:
# Crear variables adicionales de tiempo
df_encoded['month'] = df_encoded.index.month
df_encoded['day_of_week'] = df_encoded.index.day_of_week # 0=Lunes, 6=Domingo
df_encoded['is_weekend'] = df_encoded['day_of_week'].apply(lambda x: 1 if x >= 5 else 0) # Sábado y Domingo

print("\n--- DataFrame después del Feature Engineering ---")
print(df_encoded.head())


--- DataFrame después del Feature Engineering ---
                     temperature(°c)  humidity(%)  wind_speed_(m/s)  \
datetime                                                              
2017-12-01 00:00:00             -5.2           37               2.2   
2017-12-01 01:00:00             -5.5           38               0.8   
2017-12-01 02:00:00             -6.0           39               1.0   
2017-12-01 03:00:00             -6.2           40               0.9   
2017-12-01 04:00:00             -6.0           36               2.3   

                     visibility_(10m)  dew_point_temperature(°c)  \
datetime                                                           
2017-12-01 00:00:00              2000                      -17.6   
2017-12-01 01:00:00              2000                      -17.6   
2017-12-01 02:00:00              2000                      -17.7   
2017-12-01 03:00:00              2000                      -17.6   
2017-12-01 04:00:00              2000      